### Ejercicio 1:
- De la web `url = "https://books.toscrape.com"` obtén la siguiente información del primer libro:
    - Título
    - Precio
    - Stock
    - Puntuación
    - Descripción
    - Género

In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np

/tmp/ipykernel_26638/2409488756.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def get_data(url):
    response = requests.get(url)
    print(url, response.status_code)
    soup = BeautifulSoup(response.text, "html.parser")
    
    try:
        title = soup.find("h1").text.strip()
    except: title = np.nan
    
    try:
        price = soup.find("p", class_="price_color").text.strip()
    except: price = np.nan
    
    try:
        in_stock = soup.find("p", class_="instock availability").text.strip()
    except: in_stock = np.nan
    
    try:
        gender = soup.find("ul", class_="breadcrumb").find_all("li")[-2].text.strip()
    except: gender = np.nan
    
    try:
        description = soup.find("article").find("p", recursive=False).text.strip()
    except: description = np.nan
    
    try:
        rating = soup.find("p", class_="star-rating")["class"][-1]
    except: rating = np.nan
            
    return title, price, in_stock, gender, description, rating

### Ejercicio 2:
- Obtén la información que pide el **ejercicio 1** de los 20 primeros libros

_Tip: usa Selenium para navegar por la página_

### Ejercicio 3:
- Obtén la información que pide el ejercicio 1 de todos los libros que hay en la página web.

In [ ]:
base_url = "https://books.toscrape.com/catalogue/"

all_urls = []
for i in range(1,51):
    page_url = f"https://books.toscrape.com/catalogue/page-{i}.html"

    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, "html.parser")

    h3_list = soup.find_all("h3")

    urls = []
    for h3 in h3_list:
        endpoint = base_url + h3.find("a")["href"]
        urls.append(endpoint)
    
    all_urls.extend(urls)

In [ ]:
data = {
    "title" : [],
    "price" : [],
    "in_stock" :[],
    "gender" : [],
    "description" : [],
    "rating" : [],
}

for url in all_urls:
    title, price, stock, gender, description, rating = get_data(url)
    data["title"].append(title)
    data["price"].append(price)
    data["in_stock"].append(stock)
    data["gender"].append(gender)
    data["description"].append(description)
    data["rating"].append(rating)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.price = df.price.apply(lambda x: float(x[2:]))

df.in_stock = df.in_stock.apply(lambda x: int(''.join([char for char in x if char.isdigit()])))

df.rating = df.rating.replace({"One" : 1, "Two" : 2, "Three" : 3, "Four" : 4, "Five" : 5})

In [ ]:
df